In [51]:
import os
import sys
from pathlib import Path
from importlib import reload

import animaloc_improved.tools.infer_metrics as im

current_dir = Path.cwd()
if current_dir.name == "notebooks":
    os.chdir("../")
    sys.path.append("./")

In [6]:
SPECIES_MAP = {
    1: "Alcelaphinae",
    2: "Buffalo",
    3: "Kob",
    4: "Warthog",
    5: "Waterbuck",
    6: "Elephant",
}

COLORS = {
    "ground_truth": "green",
    "predictions": "red",
    "correct": "blue",
    "missed": "orange",
    "false_positive": "purple",
}

In [15]:
import pandas as pd


def get_single_image_gt(gt_df: pd.DataFrame, image_name: str) -> pd.DataFrame:
    """Extract ground truth data for a specific image."""
    image_data = gt_df[gt_df["images"] == image_name].copy()

    if image_data.empty:
        raise ValueError(f"No data found for image: {image_name}")

    print(f"Found {len(image_data)} ground truth annotations for {image_name}")
    return image_data

In [81]:
MODEL_PATH = "data/models/herdnet_v2_hn2/best_model.pth"
# MODEL_PATH = "data/models/herdnet_v2/latest_model.pth"
# gt [points]
GT_PATH = "data/gt-preprocessed/csv/test_big_size_A_B_E_K_WH_WB-points.csv"
IMAGE_ROOT = Path("data/test")
# DEVICE = "mps"
DEVICE = "mps"
THRESHOLD = 15
# gt [bboxes]
# GT_PATH = "data/groundtruth/csv/test_big_size_A_B_E_K_WH_WB-fixed-header.csv"

model = im.load_trained_model(MODEL_PATH)
gt_pt_df = pd.read_csv(GT_PATH)

Loading model from: data/models/herdnet_v2_hn2/best_model.pth


In [82]:
from torch import nn

# IMAGE = "01802f75da35434ab373569fffc1fd65a3417aef.JPG"
IMAGE = "018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG"

image_path = IMAGE_ROOT / IMAGE
ground_truth = get_single_image_gt(gt_pt_df, IMAGE)
lmds_kwargs = {"kernel_size": [3, 3], "adapt_ts": 0.3}  # mismos params de la configuración de test

reload(im)
# evaluator = im.make_evaluator( model=model, device_name=DEVICE, lmds_kwargs=lmds_kwargs )


def eval_image_v2(model: nn.Module, ground_truth: pd.DataFrame, image_path: Path, threshold: float):
    # image_path = os.path.join(image_root, image_name)
    # print(ground_truth[['images', 'x', 'y', 'labels']].to_markdown())
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    # ground_truth = get_single_image_gt(gt_, image_name)

    # Make predictions
    prediction_results = im.predict_single_image_v2(
        model=model,
        image_path=image_path,
        lmds_kwargs=lmds_kwargs,
        device=DEVICE,
    )
    predictions = prediction_results["detections"].sort_values("x")
    print(f"{len(predictions)=}")
    # print(predictions[['images', 'x', 'y', 'labels']].to_markdown())

    # Match predictions to ground truth
    matches = im.match_predictions_to_gt(predictions, ground_truth, threshold)

    # create_visualization(image_path, ground_truth, predictions, matches, COLORS, SPECIES_MAP)

    im.print_evaluation_results(matches, ground_truth, predictions, SPECIES_MAP)

    # print("\nGround Truth Data:")
    # print(ground_truth.to_string())

    # print("\nPredictions:")
    # print(predictions.to_string())

    return {
        "ground_truth": ground_truth,
        "prediction_results": prediction_results,
        "predictions": predictions,
        "matches": matches,
    }

Found 11 ground truth annotations for 018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG


In [83]:
eval_results = eval_image_v2(
    model=model,
    image_path=image_path,
    ground_truth=ground_truth,
    threshold=THRESHOLD,
)

Making predictions for: 018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG


/Users/mrestrepo/git/_personal/proyecto-de-grado/.venv/lib/python3.13/site-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
2025-11-21 12:05:51.172 | INFO     | animaloc.eval.lmds:__call__:191 - heatmap=torch.Size([1, 1, 3648, 5472]) cls_scores=torch.Size([1, 6, 3648, 5472])


 [1/1] eta: 0:00:07  time: 7.2855 data: 0.3029
 Total time: 0:00:07 (7.2877 s / it)
Found 11 predictions


/Users/mrestrepo/git/_personal/proyecto-de-grado/.venv/lib/python3.13/site-packages/animaloc/eval/metrics.py:315: RuntimeWarning: invalid value encountered in divide
  recalls = sorted_table[:,2] / n_gt


len(predictions)=11

EVALUATION RESULTS
Ground Truth Points: 11
Predicted Points: 11
True Positives: 5
False Positives: 6
False Negatives: 6
Precision: 0.455
Recall: 0.455
F1-Score: 0.455

PER-CLASS BREAKDOWN:
----------------------------------------
Elephant: GT=11, Pred=11, TP=5


In [84]:
eval_results["ground_truth"]

,images,x,y,labels
1620,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,936,1529,6
1621,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1239,1896,6
1622,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1289,1867,6
1623,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1498,1997,6
1624,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1489,2037,6
1625,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1864,1778,6
1626,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1902,1800,6
1627,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,2192,1756,6
1628,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,2146,2027,6
1629,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,2700,2395,6


In [85]:
eval_results["predictions"].sort_values("x")[["images", "x", "y", "labels"]]

,images,x,y,labels
2,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,936.0,1529.0,6
7,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1286.0,1868.0,6
0,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1718.0,658.0,6
1,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1724.0,662.0,6
4,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1866.0,1776.0,6
5,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1870.0,1782.0,6
6,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1880.0,1788.0,6
8,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,2147.0,2027.0,6
9,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,2150.0,2029.0,6
3,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,2190.0,1760.0,6


In [86]:
test_dets = pd.read_csv("data/test_results/herdnet_v2_hn2/detections.csv")
test_dets_img = test_dets[test_dets["images"] == IMAGE]
test_dets_img[["images", "x", "y", "labels"]].sort_values("x")

,images,x,y,labels
1494,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,467.0,764.0,6.0
1498,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,643.0,934.0,6.0
1492,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,859.0,329.0,6.0
1493,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,862.0,331.0,6.0
1496,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,933.0,888.0,6.0
1497,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,940.0,894.0,6.0
1499,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1073.0,1013.0,6.0
1500,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1075.0,1014.0,6.0
1495,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1095.0,880.0,6.0
1501,018f5ab5b7516a47ff2ac48a9fc08353b533c30f.JPG,1995.0,1408.0,6.0
